In [ ]:
# import libraries

import tweepy
from textblob import TextBlob
import csv
import pandas as pd
import numpy as np
import re

In [ ]:
# variables for keys associated with Twitter API - keys replaced with placeholder to prevent rate limiting

consumer_key = 'REPLACE WITH CONSUMER KEY'
consumer_key_secret = 'REPLACE WITH SECRET CONSUMER KEY'
access_token = 'REPLACE WITH ACCESS TOKEN'
access_token_secret = 'REPLACE WITH SECRET ACCESS TOKEN'

In [ ]:
# authentication

auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)

# return API with authentication

api = tweepy.API(auth)

In [ ]:
# function for cleaning tweets by removing links and special characters

def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [ ]:
# function to carry out sentiment analysis on cleaned tweet

def analize_sentiment(tweet, isPolarity):
    analysis = TextBlob(clean_tweet(tweet))
    if(isPolarity == True):
        return(analysis.polarity)
    else:
        return(analysis.subjectivity)

In [ ]:
# function to create dataframe of tweet meta data and sentiment analysis

def sentimentAnalysis(query):
    query = query
    max_tweets = 1000
    tweets = [status for status in tweepy.Cursor(api.search, q=query).items(max_tweets)]

    df = pd.DataFrame(data=[tweet.id for tweet in tweets], columns=['ID'])
    df['Date'] = np.array([tweet.created_at for tweet in tweets])
    df['Tweet'] = np.array([tweet.text for tweet in tweets])
    df['Characters'] = np.array([len(tweet.text) for tweet in tweets])
    df['Likes'] = np.array([tweet.favorite_count for tweet in tweets])
    df['Retweets'] = np.array([tweet.retweet_count for tweet in tweets])
    df['Source'] = np.array([tweet.source for tweet in tweets])
    df['Language'] = np.array([tweet.lang for tweet in tweets])
    df['Polarity'] = np.array([analize_sentiment(tweet, True) for tweet in df['Tweet']])
    df['Subjectivity'] = np.array([analize_sentiment(tweet, False) for tweet in df['Tweet']])

    return df

In [ ]:
# run function with query

sentimentAnalysis("Example")

In [ ]:
# export dataframe to csv

df.to_csv("TwitterSentimentAnalysis.csv")